In [ ]:
# Getting latest HDB resale prices from the API version of this dataset: https://data.gov.sg/collections/189/view

import requests
import time
import os

dataset_id = "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"  # make sure this is really a CKAN resource_id
base_url = "https://data.gov.sg/api/action/datastore_search"

# API key from environment variable (set in GitHub Actions secrets)
API_KEY = os.environ.get("DATAGOVSG_API_KEY")

# Retry settings
MAX_RETRIES = 5
BASE_DELAY = 10  # Base delay in seconds (API asks for 10 seconds on rate limit)

def fetch_with_retry(url, params, max_retries=MAX_RETRIES):
    """Fetch data from API with retry logic for transient failures and rate limiting."""
    headers = {}
    if API_KEY:
        headers["Authorization"] = API_KEY
        print("Using API key for authentication")
    
    for attempt in range(max_retries):
        try:
            print(f"Requesting (attempt {attempt + 1}/{max_retries})", url, params)
            resp = requests.get(url, params=params, headers=headers, timeout=60)
            
            # Handle rate limiting (429)
            if resp.status_code == 429:
                retry_after = BASE_DELAY * (attempt + 1)  # exponential backoff
                print(f"Rate limited (429). Waiting {retry_after} seconds before retry...")
                time.sleep(retry_after)
                continue
            
            # Check other HTTP errors
            if resp.status_code != 200:
                print(f"HTTP error {resp.status_code}: {resp.text[:500]}")
                if attempt < max_retries - 1:
                    delay = BASE_DELAY * (attempt + 1)
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    continue
                raise Exception(f"HTTP error {resp.status_code} after {max_retries} attempts")
            
            data = resp.json()
            
            # Check for API-level errors
            if not data.get("success", False):
                error_msg = data.get("error", {}).get("message", "Unknown API error")
                print(f"API error: {error_msg}")
                print(f"Full response: {str(data)[:500]}")
                if attempt < max_retries - 1:
                    delay = BASE_DELAY * (attempt + 1)
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    continue
                raise Exception(f"API error: {error_msg}")
            
            # Check for expected structure
            if "result" not in data:
                print(f"Unexpected API response (no 'result' key): {str(data)[:500]}")
                if attempt < max_retries - 1:
                    delay = BASE_DELAY * (attempt + 1)
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    continue
                raise Exception(f"API returned unexpected structure after {max_retries} attempts")
            
            return data
            
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}")
            if attempt < max_retries - 1:
                delay = BASE_DELAY * (attempt + 1)
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            raise
    
    raise Exception(f"Failed after {max_retries} attempts")

all_records = []
offset = 0
limit = 1000  

while True:
    params = {
        "resource_id": dataset_id,
        "limit": limit,
        "offset": offset,
    }
    
    data = fetch_with_retry(base_url, params)
    
    # grab this page of rows
    records = data["result"]["records"]

    if not records:
        break  # no more rows

    all_records.extend(records)
    
    # Delay between requests to avoid rate limiting
    time.sleep(1)

    # move to next page
    offset += limit

print("Total rows:", len(all_records))

In [ ]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# 1. Turn API records into a DataFrame and clean basic columns
#    Standard parsing, type-cleaning, and removal of unusable rows.
# -------------------------------------------------------------------
df = pd.DataFrame(all_records)

# Make sure key numeric fields are numeric
for col in ["resale_price", "floor_area_sqm", "lease_commence_date"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["month"] = pd.to_datetime(df["month"], errors="coerce")

df = df.dropna(subset=["resale_price", "floor_area_sqm", "lease_commence_date", "month"])

# -------------------------------------------------------------------
# 2. Helper: z-score within a group
#    Computes deviation of a flat from its group's mean.
#    Used only as a precursor — final scoring uses extreme-tail logic.
# -------------------------------------------------------------------

def add_group_zscore(frame, group_cols, value_col, new_col):
    grouped = frame.groupby(group_cols, observed=False)[value_col]
    mean = grouped.transform("mean")
    std = grouped.transform("std").replace(0, np.nan)
    frame[new_col] = (frame[value_col] - mean) / std
    return frame

# -------------------------------------------------------------------
# 3. PRICE SHOCK — how far above similar flats (town + flat_type)
#    Measures whether this flat sold unusually high compared to peers
#    of the same town and flat type.
#    Tiny peer groups (<5 sales) are suppressed to avoid noisy spikes.
# -------------------------------------------------------------------

df = add_group_zscore(df,
                      ["town", "flat_type"],
                      "resale_price",
                      "z_town_flat")

# -------------------------------------------------------------------
# 4. OUTLIER JUMP — block-level spike (town + block + street_name)
#    Detects sudden jumps compared to the block's own price history.
#    If historic sample size is tiny (<5), z-scores are nulled.
# -------------------------------------------------------------------

df = add_group_zscore(df,
                      ["town", "block", "street_name", "flat_type"],
                      "resale_price",
                      "z_block")

# -------------------------------------------------------------------
# 5. MARKET DEFIER — high price during a cooling month
#    Computes month-on-month median price change.
#    If the broader market is falling, high-price sales stand out.
#    'cooling_strength' captures how strongly the market was declining.
# -------------------------------------------------------------------

monthly_median = df.groupby("month", observed=False)["resale_price"].median().sort_index()
monthly_change = monthly_median.pct_change()

df = df.sort_values("month")
df["mo_change"] = df["month"].map(monthly_change)
df["cooling_strength"] = np.where(df["mo_change"] < 0, -df["mo_change"], 0)

# -------------------------------------------------------------------
# 6. UNEXPLAINABLE SPIKE — residual vs expected price model
#    Option B: Use a rolling time window for the baseline.
# -------------------------------------------------------------------

# Age of flat at point of resale (approx year-based)
df["year"] = df["month"].dt.year
df["age_years"] = df["year"] - df["lease_commence_date"]

# Coarse bins for size and age
df["size_bin"] = pd.cut(
    df["floor_area_sqm"],
    bins=[0, 40, 60, 80, 100, 130, 200],
    include_lowest=True
)

df["age_bin"] = pd.cut(
    df["age_years"],
    bins=[0, 10, 20, 30, 40, 50, 60, 80, 120],
    include_lowest=True
)

model_groups = ["town", "flat_type", "size_bin", "age_bin"]

# ---- NEW: restrict expected-price model to recent years only ----
RECENT_YEARS = 5  # adjust if needed

latest_year = df["year"].max()
cutoff_year = latest_year - RECENT_YEARS + 1

# Build baseline table first
recent_baseline = df[df["year"] >= cutoff_year].copy()

# 6a. Expected price = median of recent comparables
expected_lookup = (
    recent_baseline
    .groupby(model_groups, observed=False)["resale_price"]
    .median()
    .rename("expected_price")
    .reset_index()
)

# Merge expected_price onto df BEFORE making any mask
df = df.merge(expected_lookup, on=model_groups, how="left")

# Now residuals
df["price_residual"] = df["resale_price"] - df["expected_price"]

# ---- Recompute mask AFTER merge (this avoids the warning) ----
recent_mask = df["year"] >= cutoff_year

recent_with_resid = df[recent_mask].copy()

# 6b. Residual std from recent years
resid_std_lookup = (
    recent_with_resid
    .groupby(model_groups, observed=False)["price_residual"]
    .std()
    .replace(0, np.nan)
    .rename("resid_std")
    .reset_index()
)

# Merge std back into df
df = df.merge(resid_std_lookup, on=model_groups, how="left")

# Final z_residual
df["z_residual"] = df["price_residual"] / df["resid_std"]

# -------------------------------------------------------------------
# 7. EXTREME-TAIL SCORING (97th percentile)
#    Instead of raw z-scores, use only the extreme tail of each group.
#    Within each micro-market:
#        - Values below 97th percentile = 0
#        - Values above it are scaled between (cutoff → max)
#    This makes 100-pointers genuinely rare and highlights real outliers.
# -------------------------------------------------------------------

TAIL = 0.97

def extreme_tail(series, group_keys, tail=TAIL):
    """
    group_keys must be a list like:
        [df["town"], df["flat_type"]]
        [df["town"], df["block"], df["street_name"]]
        [df["town"], df["flat_type"], df["size_bin"], df["age_bin"]]
    """
    s = series.clip(lower=0).fillna(0)

    grouped = s.groupby(group_keys, observed=False)

    # compute 97th percentile and max within each group
    cutoff = grouped.transform(lambda x: x.quantile(tail))
    max_val = grouped.transform("max")

    # extreme tail only
    extreme = (s - cutoff).clip(lower=0)

    # denominator
    denom = (max_val - cutoff).replace(0, np.nan)

    raw = extreme / denom
    return raw.fillna(0)


# -------------------------------------------------------------------
# 7a. PRICE SHOCK SCORE
#      Extreme-tail version of z_town_flat.
# -------------------------------------------------------------------

df["price_shock_score"] = extreme_tail(
    df["z_town_flat"],
    [df["town"], df["flat_type"]]
)

# -------------------------------------------------------------------
# 7b. OUTLIER JUMP SCORE
#      Extreme-tail version of block-level deviation.
# -------------------------------------------------------------------

df["outlier_jump_score"] = extreme_tail(
    df["z_block"],
    [df["town"], df["block"], df["street_name"]]
)

# -------------------------------------------------------------------
# 7c. MARKET DEFIER SCORE
#      Uses cooling_strength × price_shock_raw,
#      then applies extreme-tail scoring by town + flat_type.
# -------------------------------------------------------------------

df["market_defier_raw"] = df["cooling_strength"] * df["z_town_flat"].clip(lower=0)

df["market_defier_score"] = extreme_tail(
    df["market_defier_raw"],
    [df["town"], df["flat_type"]]
)

# -------------------------------------------------------------------
# 7d. UNEXPLAINABLE SPIKE SCORE
#      Extreme-tail version of the residual z-score.
# -------------------------------------------------------------------

df["unexplainable_score"] = extreme_tail(
    df["z_residual"],
    [df["town"], df["flat_type"], df["size_bin"], df["age_bin"]]
)

# -------------------------------------------------------------------
# 8. WTF SCORE (0–100)
#    Editorial blend — weights preserved from original design:
#        - Price Shock      → 35%
#        - Outlier Jump     → 25%
#        - Market Defier    → 15%
#        - Unexplainable    → 25%
#    Higher scores reflect stronger evidence of a true WTF sale.
# -------------------------------------------------------------------

df["wtf_score"] = (
    df["price_shock_score"]  * 0.35 +
    df["outlier_jump_score"] * 0.25 +
    df["market_defier_score"]* 0.15 +
    df["unexplainable_score"]* 0.25
) * 100

df["wtf_score"] = df["wtf_score"].round(2)

# -------------------------------------------------------------------
# 9. WTF BUCKETS — interpretive labels
#    0–70   → Not particularly WTF
#    70-80  → Mild anomaly
#    80–85  → Something is… off
#    85–88  → Spicy deviation
#    88–100 → Feral pricing
# -------------------------------------------------------------------

def wtf_bucket(score):
    if score < 70:
        return "Not particularly WTF"
    elif score < 80:
        return "Mild anomaly"
    elif score < 85:
        return "Something is… off"
    elif score < 88:
        return "Spicy deviation"
    else:
        return "Feral pricing"

df["wtf_bucket"] = df["wtf_score"].apply(wtf_bucket)

# -------------------------------------------------------------------
# 10. Recent WTF flats (rolling 1 month)
#     Apply threshold and restrict to latest month of sales to keep
#     alerts fresh and usable for newsroom monitoring.
# -------------------------------------------------------------------

WTF_THRESHOLD = 70

wtf_flats = df[df["wtf_score"] >= WTF_THRESHOLD].copy()

latest_month = df["month"].max()
cutoff_month = latest_month - pd.DateOffset(months=1)

recent_wtf_flats = wtf_flats[wtf_flats["month"] >= cutoff_month].copy()

In [3]:
# create a new address column 

recent_wtf_flats["address"] = (
    recent_wtf_flats["block"].astype(str).str.strip() + " " +
    recent_wtf_flats["street_name"].astype(str).str.strip()
)

In [4]:
# Geocode the addresses

import geocoder

# Create empty columns
recent_wtf_flats["lat"] = None
recent_wtf_flats["lng"] = None

for idx, addr in recent_wtf_flats["address"].items():
    query = f"{addr}, Singapore"
    print("Geocoding:", query)

    g = geocoder.arcgis(query)

    if g.ok and g.latlng:
        recent_wtf_flats.at[idx, "lat"] = g.latlng[0]
        recent_wtf_flats.at[idx, "lng"] = g.latlng[1]
    else:
        print("❗Geocoding failed for:", query)

Geocoding: 652 YISHUN AVE 4, Singapore
Geocoding: 813B YISHUN RING RD, Singapore
Geocoding: 510 ANG MO KIO AVE 8, Singapore
Geocoding: 364B UPP SERANGOON RD, Singapore
Geocoding: 275C COMPASSVALE LINK, Singapore
Geocoding: 310 CANBERRA RD, Singapore
Geocoding: 832 HOUGANG CTRL, Singapore
Geocoding: 365D UPP SERANGOON RD, Singapore
Geocoding: 365B UPP SERANGOON RD, Singapore
Geocoding: 414 SERANGOON CTRL, Singapore
Geocoding: 205B COMPASSVALE LANE, Singapore
Geocoding: 641 CHOA CHU KANG ST 64, Singapore
Geocoding: 217A COMPASSVALE DR, Singapore
Geocoding: 445A CLEMENTI AVE 3, Singapore
Geocoding: 274C PUNGGOL PL, Singapore
Geocoding: 271B PUNGGOL WALK, Singapore
Geocoding: 603 ELIAS RD, Singapore
Geocoding: 530D PASIR RIS DR 1, Singapore
Geocoding: 527C PASIR RIS ST 51, Singapore
Geocoding: 325B SUMANG WALK, Singapore
Geocoding: 54 MARINE TER, Singapore
Geocoding: 271 TOH GUAN RD, Singapore
Geocoding: 101A BIDADARI PK DR, Singapore
Geocoding: 115C ALKAFF CRES, Singapore
Geocoding: 183 J

In [5]:
recent_wtf_flats = recent_wtf_flats.rename(columns={
    'wtf_bucket': 'Bucket',
    'resale_price': 'Price (S$)',
    'address': 'Location',
    'flat_type': 'Type',
    'flat_model': 'Model',
    'storey_range': 'Storey',
    'floor_area_sqm': 'Area (sqm)',
    'remaining_lease': 'Lease left'
})

In [6]:
recent_wtf_flats.to_csv('wtf_flats.csv', index=False)